# Data Prep

Mount google drive to get zip files. This will work only if dota.zip was added to your personal drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/dota.zip

Archive:  /content/drive/MyDrive/dota.zip
   creating: dota/
  inflating: __MACOSX/._dota         
   creating: dota/PGL-Arlington-Major-2022/
  inflating: __MACOSX/dota/._PGL-Arlington-Major-2022  
   creating: dota/DotA-2-The-International-2022/
  inflating: __MACOSX/dota/._DotA-2-The-International-2022  
  inflating: dota/PGL-Arlington-Major-2022/1614507_state.json  
  inflating: __MACOSX/dota/PGL-Arlington-Major-2022/._1614507_state.json  
  inflating: dota/PGL-Arlington-Major-2022/1614507_events.jsonl  
  inflating: __MACOSX/dota/PGL-Arlington-Major-2022/._1614507_events.jsonl  
  inflating: dota/PGL-Arlington-Major-2022/1597139_events.jsonl  
  inflating: __MACOSX/dota/PGL-Arlington-Major-2022/._1597139_events.jsonl  
  inflating: dota/PGL-Arlington-Major-2022/1597139_state.json  
  inflating: __MACOSX/dota/PGL-Arlington-Major-2022/._1597139_state.json  
  inflating: dota/PGL-Arlington-Major-2022/1598060_state.json  
  inflating: __MACOSX/dota/PGL-Arlington-Major-2022/._1598060_s

In [ ]:
import pandas as pd
import json


filepath = '/content/dota/DotA-2-The-International-2022/2364966_events.jsonl'

data = open(filepath)
a = 0
d = []

## creation of overall df

for line in data:
    a += 1
    d.append(json.loads(line))
df = pd.DataFrame(d)
del d

In [ ]:
data = open(filepath)
events = []
for line in data:
    a += 1
    events.append(json.loads(line)['events'][0])
events_df = pd.DataFrame(events)
print(events_df)
del events

                                        id  includesFullState  \
0     c4a7bca9-dc5c-4884-8428-343456c91e84               True   
1     9823723d-61e4-4055-bdb9-320a0a34353e               True   
2     ec785ec8-caf4-4dec-a436-288758ab174f               True   
3     ddedde12-ea22-447d-bdf5-dc9a95d580af               True   
4     c81a1416-ba30-4178-a8f7-0968041d99d9               True   
...                                    ...                ...   
7990  9a521b66-0514-4a32-b19c-133f233ee7d9               True   
7991  50181c6a-926c-4306-97eb-abdd13a9f391               True   
7992  cd698d6e-a217-4a14-b297-5cc10721fe60               True   
7993  873d7ab3-496d-4af5-bc6e-ed432842083b               True   
7994  49a2fcc1-e3f7-4326-98a5-e617b405ce3f               True   

                                type  \
0                  grid-started-feed   
1          tournament-started-series   
2              team-banned-character   
3              team-banned-character   
4              team

In [ ]:
## creation of destroyed tower df

i = 1 # iterator starts from 1 bc dataframes start from 1 (there's an error when starting from 0)
dt_list = [] # list of destroyed towers + the ENTRY id + the time occurred
logged_list = [] # list of towers that have already been logged
occurredTime = []

while i < len(df.index):
  for j in range(38): # 38 is the number of towers (should make dynamic later on)
    try:
      tower = events_df.iloc[i]["seriesState"]["games"][0]["structures"][j]
      #print("tower {0} destroyed status is {1} at {2}, id is: {3}".format(j, tower["destroyed"], df.iloc[i]["occurredAt"], df.iloc[i]["id"]))

      if (tower["destroyed"] == True) and ("tower{0}".format(j) not in logged_list): #checking if tower is destroyed and if it exists in dt_list already
        occurredTime.append(df.iloc[i]["occurredAt"])
        dt_list.append({'EVENT ID' : df.iloc[i]["id"],
                        'TOWER ID' : tower["id"]})
        logged_list.append("tower{0}".format(j))
      else:
        continue

    except IndexError:
      print("line {0} doesn't have towers yet".format(i))

    finally:
      i +=1

dt_df = pd.DataFrame({'timeOccurred': occurredTime, 'tower_df' : dt_list}) # THIS BOI IS THE DATAFRAME
print(dt_df)

line 1 doesn't have towers yet
line 2 doesn't have towers yet
line 3 doesn't have towers yet
line 4 doesn't have towers yet
line 5 doesn't have towers yet
line 6 doesn't have towers yet
line 7 doesn't have towers yet
line 8 doesn't have towers yet
line 9 doesn't have towers yet
line 10 doesn't have towers yet
line 11 doesn't have towers yet
line 12 doesn't have towers yet
line 13 doesn't have towers yet
line 14 doesn't have towers yet
line 15 doesn't have towers yet
line 16 doesn't have towers yet
line 17 doesn't have towers yet
line 18 doesn't have towers yet
line 19 doesn't have towers yet
line 20 doesn't have towers yet
line 21 doesn't have towers yet
line 22 doesn't have towers yet
line 23 doesn't have towers yet
line 24 doesn't have towers yet
line 25 doesn't have towers yet
line 26 doesn't have towers yet
line 27 doesn't have towers yet
line 28 doesn't have towers yet
line 29 doesn't have towers yet
line 30 doesn't have towers yet
line 31 doesn't have towers yet
line 32 doesn't h

In [ ]:
# initializing and updating player cards with each iteration (ADDS *EVERYTHING* TO PLAYERS LIST)

players_list = []
player_list = []
processed_players = []
timeOccurred = []
i = 1

while i <=30:
    try:
      teams = events_df.iloc[i]["seriesState"]["games"][0]["teams"] #making
      timeOccurred.append(df.iloc[i]["occurredAt"])

      for team in teams:
        players = team["players"]

        for player in players:
          item_list = []

          for item in player["items"]:
            if item["equipped"]:
              item_list.append(item['id'])


          player_list.append({
            'id' : player["id"],
            'name' : player["name"],
            'team' :  team["name"],
            'character' : player["character"]["name"],
            'netWorth' : player["netWorth"],
            'items' : item_list,
            'lifeStatus' : player["alive"],
            'kills' : player["kills"],
            'deaths' : player["deaths"]})
      players_list.append(player_list)
      print(players_list)
    except IndexError:
      print("line {0} doesn't have players yet".format(i))

    finally:
      i +=1

#player_df = pd.DataFrame({'timeOccurred':timeOccurred, 'player_df' : players_list})
#print(player_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
entire_df = player_df.join(dt_df.set_index('timeOccurred'), on = 'timeOccurred')
print(entire_df)
entire_df.to_json(r'/content/drive/MyDrive/export_dataframe.json')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



NameError: ignored